In [8]:
### RAG Pipelines - Data Ingestion to Vector DB

import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

from pathlib import Path

In [9]:
### Read all the pdfs inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data/text_files")

Found 4 PDF files to process

Processing: Active Armed Offender.pdf
  ✓ Loaded 9 pages

Processing: Armed Intruder & Hostage (including Siege).pdf
  ✓ Loaded 3 pages

Processing: OSP12.1 Mobility Equipment Procedure 1.pdf
  ✓ Loaded 8 pages

Processing: Physical Barrier Management.pdf
  ✓ Loaded 2 pages

Total documents loaded: 22


In [10]:
all_pdf_documents

[Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-09-30T08:40:59+10:00', 'author': 'Radhika Lal', 'moddate': '2025-09-30T08:40:59+10:00', 'title': 'Emergency Response Procedures (Red Book) 2025 08.pdf', 'source': '..\\data\\text_files\\New folder\\Active Armed Offender.pdf', 'total_pages': 9, 'page': 0, 'page_label': '1', 'source_file': 'Active Armed Offender.pdf', 'file_type': 'pdf'}, page_content=''),
 Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'creationdate': '2025-09-30T08:40:59+10:00', 'author': 'Radhika Lal', 'moddate': '2025-09-30T08:40:59+10:00', 'title': 'Emergency Response Procedures (Red Book) 2025 08.pdf', 'source': '..\\data\\text_files\\New folder\\Active Armed Offender.pdf', 'total_pages': 9, 'page': 1, 'page_label': '2', 'source_file': 'Active Armed Offender.pdf', 'file_type': 'pdf'}, page_content=''),
 Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': 'PyPDF', 'crea

In [11]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs    

In [12]:
chunks=split_documents(all_pdf_documents)
chunks

Split 22 documents into 12 chunks

Example chunk:
Content: CCD-SMS-OSP-12.1 Mobility Equipment Procedure. v1.0. CCD Risk & Compliance.  
Printed versions of this document may not be current. Check CCD Hub for the latest version of this document. 1 
 
 
 
 
 
...
Metadata: {'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-07-04T16:32:14+10:00', 'title': 'CX Safety', 'author': 'Belinda Carr', 'moddate': '2024-07-04T16:32:14+10:00', 'source': '..\\data\\text_files\\New folder\\OSP12.1 Mobility Equipment Procedure 1.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': 'OSP12.1 Mobility Equipment Procedure 1.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-07-04T16:32:14+10:00', 'title': 'CX Safety', 'author': 'Belinda Carr', 'moddate': '2024-07-04T16:32:14+10:00', 'source': '..\\data\\text_files\\New folder\\OSP12.1 Mobility Equipment Procedure 1.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': 'OSP12.1 Mobility Equipment Procedure 1.pdf', 'file_type': 'pdf'}, page_content='CCD-SMS-OSP-12.1 Mobility Equipment Procedure. v1.0. CCD Risk & Compliance.  \nPrinted versions of this document may not be current. Check CCD Hub for the latest version of this document. 1 \n \n \n \n \n \n \n• \n• \n• \n•'),
 Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-07-04T16:32:14+10:00', 'title': 'CX Safety', 'author': 'Belinda Carr', 'moddate': '2024-07-04T16:32:14+10:00', 'source': '..\\data\\text_files\\New fo

In [13]:
### embedding and vectorstore DB
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [14]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [15]:
### Vector Store
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 12


In [16]:
chunks

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-07-04T16:32:14+10:00', 'title': 'CX Safety', 'author': 'Belinda Carr', 'moddate': '2024-07-04T16:32:14+10:00', 'source': '..\\data\\text_files\\New folder\\OSP12.1 Mobility Equipment Procedure 1.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1', 'source_file': 'OSP12.1 Mobility Equipment Procedure 1.pdf', 'file_type': 'pdf'}, page_content='CCD-SMS-OSP-12.1 Mobility Equipment Procedure. v1.0. CCD Risk & Compliance.  \nPrinted versions of this document may not be current. Check CCD Hub for the latest version of this document. 1 \n \n \n \n \n \n \n• \n• \n• \n•'),
 Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2024-07-04T16:32:14+10:00', 'title': 'CX Safety', 'author': 'Belinda Carr', 'moddate': '2024-07-04T16:32:14+10:00', 'source': '..\\data\\text_files\\New fo

In [17]:
### convert the text to embeddings
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 12 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  1.43it/s]

Generated embeddings with shape: (12, 384)
Adding 12 documents to vector store...
Successfully added 12 documents to vector store
Total documents in collection: 24


### Retriever Pipeline from VectorStore

In [18]:
class RAGRetriever:
   """Handles query-based retrieval from the vector store"""
    
   def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

   def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                   # similarity_score = 1 - distance
                    similarity_score = 1.0 / (1.0 + distance)
                    print(f"Doc {i+1}: distance={distance}, similarity_score={similarity_score}")
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [19]:
rag_retriever

In [20]:
rag_retriever.retrieve("What are Physical Barriers?")

Retrieving documents for query: 'What are Physical Barriers?'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 69.23it/s]

Generated embeddings with shape: (1, 384)
Doc 1: distance=1.03011155128479, similarity_score=0.4925837692845663
Doc 2: distance=1.03011155128479, similarity_score=0.4925837692845663
Doc 3: distance=1.3534599542617798, similarity_score=0.42490631641687504
Doc 4: distance=1.3534599542617798, similarity_score=0.42490631641687504
Doc 5: distance=1.5765080451965332, similarity_score=0.38812221132564756
Retrieved 5 documents (after filtering)


[{'id': 'doc_a5516cc4_8',
  'content': 'CX Safety Physical Barrier Management \n \nSafety Policies and Procedures \nSOP Title & \nWork Stream PHYSICAL BARRIER MANAGEMENT \nUpdated V1.0 – April 19 – CX Risk & Security \nPurpose To ensure the effectiveness of operation of all physical security barriers. \nDescription / \nBackground \nPhysical security barriers will be operated and maintained in accordance with \nmanufacturer’s specifications and in such a manner to ensure the highest \nstandard of operational performance. \nPROCEDURE \nThe Centre Manager and Risk and Security Manager will ensure the \neffectiveness of operation of all physical security barriers including but not \nlimited to: \n• Public entry/exit doors and shutters \n• Fire passage and fire stairwell perimeter doors \n• Dock roller shutters, gates and grilles \n• Roof hatches and external or perimeter service doors \n• Car park gates, shutters and grilles \n• Car park ramp swing gates and booms \n• Car park and other pe

In [21]:
rag_retriever.retrieve("Active Armed Offender")

Retrieving documents for query: 'Active Armed Offender'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 87.13it/s]

Generated embeddings with shape: (1, 384)
Doc 1: distance=1.6184139251708984, similarity_score=0.381910587316607
Doc 2: distance=1.6184139251708984, similarity_score=0.381910587316607
Doc 3: distance=1.6973414421081543, similarity_score=0.3707354153942159
Doc 4: distance=1.6973414421081543, similarity_score=0.3707354153942159
Doc 5: distance=1.770911693572998, similarity_score=0.3608920494721842
Retrieved 5 documents (after filtering)


[{'id': 'doc_29147740_9',
  'content': '• Roof hatches and external or perimeter service doors \n• Car park gates, shutters and grilles \n• Car park ramp swing gates and booms \n• Car park and other perimeter fences, gates and bollards \n• All perimeter and internal security locks and padlocks \n• Demountable internal access control security grilles and barriers, \nbarricades. \n• Localised portable electronic alert systems \n \nThe Security Manager will ensure: \n• That the location and type of each physical security barrier is marked \non a set of plans available in the Security Office and the Critical \nInfrastructure Dashboard.  This may be coupled with plans also \ndisplaying other security installations. \n• That the computer maintenance system or similar is used to record daily \nfaults, failures and defects in the physical security barriers and to initiate \naction to effect repair/rectification of inoperative or malfunctioning \nequipment. \n• That a daily checklist is impleme

### Integration Vector DB Context Pipleine with LLM Output

In [ ]:
### Simple RAG Pipleline with Groq LLM
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv
load_dotenv()


## initialise the groq LLM
#groq_API_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(groq_api_key=os.getenv("GROQ_API_KEY"),model_name="llama-3.1-8b-instant",temperature=0.1,max_tokens=1024)

#Simple RAG Function
def rag_simple(query,retriever,llm,top_k:int=3):
    ## retriever the context
    results=retriever.retrieve(query,top_k=top_k)
    context="\n\n".join([doc['content'] for doc in results]) if results else ""
    if not context:
        return "No relevant context found to answer the question."
    
    ## generate the answwer using GROQ LLM
    prompt=f"""Use the following context to answer the question concisely.
        Context:
        {context}

        Question: {query}

        Answer:""".strip()
    response = llm.invoke(prompt)
   # response=llm.invoke([prompt.format(context=context,query=query)])
    return response.content

In [39]:
answer= rag_simple("What is the definiton of barrier?",rag_retriever,llm)
print(answer)

Retrieving documents for query: 'What is the definiton of barrier?'
Top K: 3, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.92it/s]

Generated embeddings with shape: (1, 384)
Doc 1: distance=1.0766974687576294, similarity_score=0.4815337886448349
Doc 2: distance=1.0766974687576294, similarity_score=0.4815337886448349
Doc 3: distance=1.4173791408538818, similarity_score=0.41367114620124246
Retrieved 3 documents (after filtering)


The context does not explicitly define the term "barrier." However, based on the context, a barrier can be inferred to refer to any physical security feature or device used to restrict or control access to a particular area, such as doors, shutters, gates, grilles, fences, bollards, and other similar security installations.


### Enhanced RAG Pipeline Features

In [41]:
def rag_advanced(query, retriever, llm, top_k=5, min_score=0.2, return_context=False):
    """
    RAG pipeline with extra features:
    - Returns answer, sources, confidence score, and optionally full context.
    """
    results = retriever.retrieve(query, top_k=top_k, score_threshold=min_score)
    if not results:
        return {'answer': 'No relevant context found.', 'sources': [], 'confidence': 0.0, 'context': ''}
    
    # Prepare context and sources
    context = "\n\n".join([doc['content'] for doc in results])
    sources = [{
        'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
        'page': doc['metadata'].get('page', 'unknown'),
        'score': doc['similarity_score'],
        'preview': doc['content'][:300] + '...'
    } for doc in results]
    confidence = max([doc['similarity_score'] for doc in results])
    
    # Generate answer
    prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {query}\n\nAnswer:"""
    response = llm.invoke([prompt.format(context=context, query=query)])
    
    output = {
        'answer': response.content,
        'sources': sources,
        'confidence': confidence
    }
    if return_context:
        output['context'] = context
    return output

# Example usage:
result = rag_advanced("How Physical Barriers are placed?", rag_retriever, llm, top_k=3, min_score=0.1, return_context=True)
print("Answer:", result['answer'])
print("Sources:", result['sources'])
print("Confidence:", result['confidence'])
print("Context Preview:", result['context'][:300])

Retrieving documents for query: 'How Physical Barriers are placed?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 57.64it/s]

Generated embeddings with shape: (1, 384)
Doc 1: distance=0.9675390124320984, similarity_score=0.508249134416851
Doc 2: distance=0.9675390124320984, similarity_score=0.508249134416851
Doc 3: distance=1.3451844453811646, similarity_score=0.42640569357753405
Retrieved 3 documents (after filtering)


Answer: According to the provided context, Physical Barriers are placed in accordance with manufacturer's specifications and in such a manner to ensure the highest standard of operational performance.
Sources: [{'source': 'Physical Barrier Management.pdf', 'page': 0, 'score': 0.508249134416851, 'preview': 'CX Safety Physical Barrier Management \n \nSafety Policies and Procedures \nSOP Title & \nWork Stream PHYSICAL BARRIER MANAGEMENT \nUpdated V1.0 – April 19 – CX Risk & Security \nPurpose To ensure the effectiveness of operation of all physical security barriers. \nDescription / \nBackground \nPhysical securi...'}, {'source': 'Physical Barrier Management.pdf', 'page': 0, 'score': 0.508249134416851, 'preview': 'CX Safety Physical Barrier Management \n \nSafety Policies and Procedures \nSOP Title & \nWork Stream PHYSICAL BARRIER MANAGEMENT \nUpdated V1.0 – April 19 – CX Risk & Security \nPurpose To ensure the effectiveness of operation of all physical security barriers. \nDescription / 

In [42]:
# --- Advanced RAG Pipeline: Streaming, Citations, History, Summarization ---
from typing import List, Dict, Any
import time

class AdvancedRAGPipeline:
    def __init__(self, retriever, llm):
        self.retriever = retriever
        self.llm = llm
        self.history = []  # Store query history

    def query(self, question: str, top_k: int = 5, min_score: float = 0.2, stream: bool = False, summarize: bool = False) -> Dict[str, Any]:
        # Retrieve relevant documents
        results = self.retriever.retrieve(question, top_k=top_k, score_threshold=min_score)
        if not results:
            answer = "No relevant context found."
            sources = []
            context = ""
        else:
            context = "\n\n".join([doc['content'] for doc in results])
            sources = [{
                'source': doc['metadata'].get('source_file', doc['metadata'].get('source', 'unknown')),
                'page': doc['metadata'].get('page', 'unknown'),
                'score': doc['similarity_score'],
                'preview': doc['content'][:120] + '...'
            } for doc in results]
            # Streaming answer simulation
            prompt = f"""Use the following context to answer the question concisely.\nContext:\n{context}\n\nQuestion: {question}\n\nAnswer:"""
            if stream:
                print("Streaming answer:")
                for i in range(0, len(prompt), 80):
                    print(prompt[i:i+80], end='', flush=True)
                    time.sleep(0.05)
                print()
            response = self.llm.invoke([prompt.format(context=context, question=question)])
            answer = response.content

        # Add citations to answer
        citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i, src in enumerate(sources)]
        answer_with_citations = answer + "\n\nCitations:\n" + "\n".join(citations) if citations else answer

        # Optionally summarize answer
        summary = None
        if summarize and answer:
            summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
            summary_resp = self.llm.invoke([summary_prompt])
            summary = summary_resp.content

        # Store query history
        self.history.append({
            'question': question,
            'answer': answer,
            'sources': sources,
            'summary': summary
        })

        return {
            'question': question,
            'answer': answer_with_citations,
            'sources': sources,
            'summary': summary,
            'history': self.history
        }

# Example usage:
adv_rag = AdvancedRAGPipeline(rag_retriever, llm)
result = adv_rag.query("How Physical Barriers are placed?", top_k=3, min_score=0.1, stream=True, summarize=True)
print("\nFinal Answer:", result['answer'])
print("Summary:", result['summary'])
print("History:", result['history'][-1])

Retrieving documents for query: 'How Physical Barriers are placed?'
Top K: 3, Score threshold: 0.1
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 54.50it/s]

Generated embeddings with shape: (1, 384)
Doc 1: distance=0.9675390124320984, similarity_score=0.508249134416851
Doc 2: distance=0.9675390124320984, similarity_score=0.508249134416851
Doc 3: distance=1.3451844453811646, similarity_score=0.42640569357753405
Retrieved 3 documents (after filtering)
Streaming answer:
Use the following context to answer the question concisely.
Context:
CX Safety Physical Barrier Management 
 
Safety Policies and Procedures 
SOP Title & 
Work Stream PHYSICAL BARRIER MANAGEMENT 
Updated V1.0 – April 19 – CX Risk & Security 
Purpose To ensure the effectiveness of operation of all physical security barr

iers. 
Description / 
Background 
Physical security barriers will be operated and maintained in accordance with 
manufacturer’s specifications and in such a manner to ensure the highest 
standard of operational performance. 
PROCEDURE 
The Centre Manager and Risk and Security Manager will ensure the 
effectiveness of operation of all physical security barriers including but not 
limited to: 
• Public entry/exit doors and shutters 
• Fire passage and fire stairwell perimeter doors 
• Dock roller shutters, gates and grilles 
• Roof hatches and external or perimeter service doors 
• Car park gates, shutters and grilles 
• Car park ramp swing gates and booms 
• Car park and other perimeter fences, gates and bollards

CX Safety Physical Barrier Management 
 
Safety Policies and Procedures 
SOP Title & 
Work Stream PHYSICAL BARRIER MANAGEMENT 
Updated V1.0 – April 19 – CX Risk & Security 
Purpose To ensure the effectiveness of operation of all physical security barriers. 
Description / 
Back